In [10]:
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np

# target url
url = 'https://go.drugbank.com/'
# for ATC 
# url = 'https://go.drugbank.com/atc'


In [2]:
# Drug bank common name list
import pandas as pd

df = pd.read_csv('D:/WorkSpace/master_thesis/Drug_Bank/drugbank_vocabulary.csv')
df['ATC'] = 0
drugName = df['Common name']


In [11]:
# webdriver setting
# driver.quit()
# Chromedriver
driver = webdriver.Chrome(executable_path=r'D:\WorkSpace\master_thesis\chromedriver_win32\chromedriver.exe')
driver.get(url)


# Phantomjs
# driver = webdriver.PhantomJS(executable_path=r'D:\WorkSpace\phantomjs-2.1.1-windows\bin\phantomjs.exe')
# driver.get(url)



In [12]:
# from urllib.request import FancyURLopener
     
# temp = AppURLopener().open(url)

NameError: name 'AppURLopener' is not defined

In [ ]:
# ATC Classificatio page
from urllib.request import FancyURLopener

for i in range(1564, len(drugName)):
# for i in range(3):
    # search with drug name
    try:
        search_window  = driver.find_element_by_xpath('//*[@id="query"]')
        search_window.send_keys(drugName[i])
        search_window.send_keys(Keys.ENTER)
    except:
        print("Re Connection")
        driver.quit()
#         driver = webdriver.PhantomJS(executable_path=r'D:\WorkSpace\phantomjs-2.1.1-windows\bin\phantomjs.exe')
        driver = webdriver.Chrome(executable_path=r'D:\WorkSpace\master_thesis\chromedriver_win32\chromedriver.exe')
        driver.get(url)
    
    # Scraping ATC Code
    try:
        atcData = driver.find_element_by_css_selector("body > main > div > div > div.drug-card > div.card-content.px-md-4.px-sm-2.pb-md-4.pb-sm-2 > dl:nth-child(10) > dd:nth-child(2) > a")
        atcCode = atcData.text.split()[0]

        df['ATC'][i] = atcCode
    except: # ACT code가 없을 경우
        df['ATC'][i] = np.NaN
    

<ipython-input-4-27790ac0b981>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ATC'][i] = np.NaN
C:\Users\paten\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Re Connection
Re Connection
Re Connection
Re Connection
Re Connection


In [13]:
# Drug name으로 검색 
search_window  = driver.find_element_by_xpath('//*[@id="query"]')
search_window.send_keys(drugName[1596])
search_window.send_keys(Keys.ENTER)

In [9]:
drugName[1566]

'{(4Z)-2-[(1R,2R)-1-Amino-2-hydroxypropyl]-4-[(4-amino-1H-indol-3-yl)methylene]-5-oxo-4,5-dihydro-1H-imidazol-1-yl}acetic acid'

In [31]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

atc_code = soup.select_one('body > main > div > div > div.drug-card > div.card-content.px-md-4.px-sm-2.pb-md-4.pb-sm-2')

body > main > div > div > div.drug-card > div.card-content.px-md-4.px-sm-2.pb-md-4.pb-sm-2 > dl:nth-child(10) > dd:nth-child(2) > a

In [34]:
type(atc_code)
# atcCode = atc_code.text
# print(type(atcCode))

bs4.element.Tag

In [6]:
def parsing_Data(html, rank):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    title = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2')
    author = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth')
    try:
        author.span.extract()
        author = author.text.strip()
    except:
        author = author.text.split('글/')[0].strip()
        if author.find('원저/') != -1:
            author = author.split('원저/')[1].strip()
        else:
            author = author.split('저')[0].strip()
    
    publisher = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_pub > a')
    publication = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_date')
    price = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > div.gd_infoTb').find('tr', {'class':'accentRow'}).find('em', {'class':'yes_m'}).text
    
    try:
        review = soup.select_one('#infoset_reviewContentList').find('div', {'class':'review_cont'}).text
        if review.find('.') != -1:
            posPeriod  = review.find('.')
        else:
            posPeriod = 10000000000

        if review.find('?') != -1:
            posQuestion  = review.find('?')
        else:
            posQuestion = 10000000000

        if posPeriod < posQuestion:
            punctuation = '.'
        elif posPeriod > posQuestion:
            punctuation ='?'
        else:
            punctuation =''

        review = re.split('(?<=[\D])[\.|\n|\?]', review)[1].strip()
        review = review + punctuation        
    except:
        review = 'N/A'  

    infoText = str(rank+1) + '. 제목: ' + title.text + ', 저자: ' + author + ', 출판사: ' + publisher.text.replace('\n', '') + ', 판매가: ' + price + '원, 출간일: ' + publication.text
    
    f.write(infoText+'\n')
    f.write(review+'\n')
    
        

In [7]:
bestSeller=driver.find_element_by_xpath('//*[@id="yesFixCorner"]/dl/dd/ul[1]/li[1]/a')
bestSeller.click()

# Get Rank 1~10 Xpath
rankXpath = [0]*10
for i in range(10):
    rankXpath[i] = '//*[@id="bestList"]/ol/li[' + str(i+1) + ']/p[1]/a'

In [8]:
f = open('yes24_bestSeller_Top10.txt', 'w', encoding = 'utf8')
    
for i in range(len(rankXpath)):
    driver.implicitly_wait(1)
    rankElement = driver.find_element_by_xpath(rankXpath[i])
    #rankElement.click()
    rankElement.send_keys(Keys.ENTER)
    driver.implicitly_wait(1)
    html = driver.page_source
    driver.implicitly_wait(1)
    parsing_Data(html, i)
    driver.back()
    
    
f.close()

AttributeError: 'NoneType' object has no attribute 'text'

In [6]:
#pagq별 테스트
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

#title = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2')
title = soup.find('h2', {'class':'gd_name'})    
author = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth')

try:
    author.span.extract()
    author = author.text.strip()
except:
    author = author.text.split('글/')[0].strip()
    if author.find('원저/') != -1:
        author = author.split('원저/')[1].strip()
    else:
        author = author.split('저')[0].strip()
'''
try:
    author.span.extract()
    author = author.text.strip()
except:
    author = author.text.split('글/')[0].strip()
    if author.find('원저/') != -1:
        author = author.split('원저/')[1].strip()
    else:
        author = author.split('저')[0].strip()
'''

publisher = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_pub > a')
publication = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_date')
price = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > div.gd_infoTb').find('tr', {'class':'accentRow'}).find('em', {'class':'yes_m'}).text

try:
    review = soup.select_one('#infoset_reviewContentList').find('div', {'class':'review_cont'}).text
    if review.find('.') != -1:
        posPeriod  = review.find('.')
    else:
        posPeriod = 10000000000

    if review.find('?') != -1:
        posQuestion  = review.find('?')
    else:
        posQuestion = 10000000000

    if posPeriod < posQuestion:
        punctuation = '.'
    elif posPeriod > posQuestion:
        punctuation ='?'
    else:
        punctuation =''

    review = re.split('(?<=[\D])[\.|\n|\?]', review)[1].strip()
    if len(review) > 50:
        review = re.split(r'다 ', review)[0] + '다.'
    else:
        review = review + punctuation        
except:
    review = 'N/A'  

infoText = '1. 제목: ' + title.text + ', 저자: ' + author + ', 출판사: ' + publisher.text.replace('\n', '') + ', 판매가: ' + price + '원, 출간일: ' + publication.text

print(infoText)
print(review)

1. 제목: 마음챙김의 시, 저자: 류시화 편, 출판사: 수오서재, 판매가: 11,700원, 출간일: 2020년 09월 17일
시를 읽는 것은 자기 자신으로 돌아오는 것이고 세상을 경이롭게 여기는 것이며 여러 색의 감정을 경험하는 것이다.
